In [1]:
# datasets
# rugby-balls: https://universe.roboflow.com/hotchickenpie/ruby-ball-detection/browse?queryText=&pageSize=50&startingIndex=0&browseQuery=true
# mini-rugby balls: https://universe.roboflow.com/muhie-kk0uz/rugby-balls-zcnjx
# ping-pong balls: https://universe.roboflow.com/pingpong-ojuhj/ping-pong-detection-0guzq/images/8SLEQwQoyRiRvY70azBS
# our custom simulation ping pong balls & mini rugby balls dataset: 

In [11]:
# !pip install python-dotenv
# !pip install roboflow

In [12]:
from roboflow import Roboflow
from dotenv import load_dotenv
import os

In [13]:
load_dotenv()
api_key = os.getenv("API_KEY")
api_key2 = os.getenv("API_KEY2")
api_key3 = os.getenv("API_KEY3")

In [17]:
rf = Roboflow(api_key=api_key3)
project = rf.workspace("webotspingpongballsminirugbyballs").project("webotspingpongballsminirugby")
version = project.version(2)
dataset = version.download("yolov12")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to WeBotsPingpongballsMiniRugby-2 in yolov12:: 100%|██████████| 455/455 [00:00<00:00, 1606.11it/s]


In [3]:
rf = Roboflow(api_key=api_key)
project = rf.workspace("hotchickenpie").project("ruby-ball-detection")
version = project.version(3)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Ruby-Ball-Detection-3 in yolov11:: 100%|██████████| 1304/1304 [00:01<00:00, 1291.36it/s]


In [4]:
rf = Roboflow(api_key=api_key)
project = rf.workspace("pingpong-ojuhj").project("ping-pong-detection-0guzq")
version = project.version(3)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Ping-Pong-Detection-3 in yolov11:: 100%|██████████| 49688/49688 [00:32<00:00, 1506.55it/s]


# Merge Datasets

In [2]:
# !pip install roboflow

  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [7]:
# Import necessary libraries
import os
import shutil
import glob
import random
import logging
import yaml
from roboflow import Roboflow

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
API_KEY = api_key
OUTPUT_DIR = r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Computer_Vision\datasets"

# Define datasets with unique variable names
datasets_info = [
    {
        "workspace": "hotchickenpie",
        "project": "ruby-ball-detection",
        "version": 3,
        "download_format": "yolov11",
        "local_path": r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\unibots\src\Ruby-Ball-Detection-3"
    },
    {
        "workspace": "pingpong-ojuhj",
        "project": "ping-pong-detection-0guzq",
        "version": 3,
        "download_format": "yolov11",
        "local_path": r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\unibots\src\Ping-Pong-Detection-3"
    }
]

# Initialize Roboflow
rf = Roboflow(api_key=API_KEY)

# Download each dataset
for dataset in datasets_info:
    project = rf.workspace(dataset["workspace"]).project(dataset["project"])
    version = project.version(dataset["version"])
    logging.info(f"Downloading dataset '{dataset['project']}' version {dataset['version']}...")
    version.download(dataset["download_format"], location=dataset["local_path"])
    logging.info(f"Downloaded to '{dataset['local_path']}'")

# Define source datasets paths
SOURCE_DATASETS = [dataset["local_path"] for dataset in datasets_info]

# Define splits to ensure a total of 200 images per dataset
# Example split: 160 train, 30 valid, 10 test
SPLITS = {
    "train": 550,
    "valid": 50,
    "test": 34
}

OUTPUT_IMAGES_DIR = os.path.join(OUTPUT_DIR, "images")
OUTPUT_LABELS_DIR = os.path.join(OUTPUT_DIR, "labels")
OUTPUT_DATA_YAML = os.path.join(OUTPUT_DIR, "data.yaml")

# Create output directories
for split in SPLITS.keys():
    os.makedirs(os.path.join(OUTPUT_IMAGES_DIR, split), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_LABELS_DIR, split), exist_ok=True)

# Function to read data.yaml
def read_data_yaml(dataset_path):
    data_yaml_path = os.path.join(dataset_path, "data.yaml")
    if not os.path.exists(data_yaml_path):
        logging.error(f"data.yaml not found in {dataset_path}")
        return None
    with open(data_yaml_path, 'r') as f:
        data = yaml.safe_load(f)
    return data

# Function to merge class names
def merge_classes(datasets_data):
    merged_names = []
    class_map = {}  # {dataset_idx: {original_class: merged_class_index}}
    
    for idx, data in enumerate(datasets_data):
        if data is None:
            continue  # Skip datasets without data.yaml
        class_map[idx] = {}
        for cls in data['names']:
            if cls not in merged_names:
                merged_names.append(cls)
            class_map[idx][cls] = merged_names.index(cls)
    
    return merged_names, class_map

# Read and merge data.yaml files
datasets_data = [read_data_yaml(ds) for ds in SOURCE_DATASETS]
merged_class_names, class_maps = merge_classes(datasets_data)
merged_nc = len(merged_class_names)

# Create merged data.yaml
merged_data_yaml = {
    'train': os.path.join(OUTPUT_IMAGES_DIR, 'train'),
    'val': os.path.join(OUTPUT_IMAGES_DIR, 'valid'),
    'test': os.path.join(OUTPUT_IMAGES_DIR, 'test'),
    'nc': merged_nc,
    'names': merged_class_names
}

with open(OUTPUT_DATA_YAML, 'w') as f:
    yaml.dump(merged_data_yaml, f)

logging.info(f"Merged data.yaml created at {OUTPUT_DATA_YAML}")

def select_and_copy(dataset_idx, dataset_path, split, num_images, split_type):
    split_path = os.path.join(dataset_path, split_type)
    
    if not os.path.exists(split_path):
        logging.warning(f"Split folder '{split_type}' does not exist in '{dataset_path}'. Skipping.")
        return

    images_path = os.path.join(split_path, "images")
    labels_path = os.path.join(split_path, "labels")

    if not os.path.exists(images_path):
        logging.warning(f"'images' folder does not exist in '{split_path}'. Skipping.")
        return
    if not os.path.exists(labels_path):
        logging.warning(f"'labels' folder does not exist in '{split_path}'. Skipping.")
        return

    # Get all .jpg and .png images
    image_files = glob.glob(os.path.join(images_path, "*.jpg")) + glob.glob(os.path.join(images_path, "*.png"))

    if not image_files:
        logging.warning(f"No image files found in '{images_path}'.")
        return

    # Sort image files to take the first N
    image_files.sort()
    selected_image_files = image_files[:num_images]

    # Get corresponding label files
    for img_file in selected_image_files:
        basename = os.path.splitext(os.path.basename(img_file))[0]
        src_label = os.path.join(labels_path, f"{basename}.txt")

        if not os.path.exists(src_label):
            logging.warning(f"Label file '{src_label}' does not exist. Skipping image '{basename}'.")
            continue

        # Destination paths
        dest_image = os.path.join(OUTPUT_IMAGES_DIR, split, os.path.basename(img_file))
        dest_label = os.path.join(OUTPUT_LABELS_DIR, split, os.path.basename(src_label))

        # Copy image
        shutil.copy2(img_file, dest_image)

        # Read label, remap class if necessary
        with open(src_label, 'r') as f:
            lines = f.readlines()

        remapped_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                logging.warning(f"Unexpected label format in '{src_label}'. Skipping line.")
                continue
            original_cls, x_center, y_center, width, height = parts
            try:
                original_cls = int(original_cls)
            except ValueError:
                logging.warning(f"Invalid class index in '{src_label}': '{original_cls}'. Skipping line.")
                continue
            # Map to merged class
            if datasets_data[dataset_idx] is None:
                logging.warning(f"No data.yaml found for dataset index {dataset_idx}. Cannot remap classes.")
                remapped_cls = original_cls
            else:
                cls_name = datasets_data[dataset_idx]['names'][original_cls]
                remapped_cls = class_maps[dataset_idx].get(cls_name, original_cls)
            remapped_line = f"{remapped_cls} {x_center} {y_center} {width} {height}\n"
            remapped_lines.append(remapped_line)

        # Write remapped label
        with open(dest_label, 'w') as f:
            f.writelines(remapped_lines)

    logging.info(f"Copied {len(selected_image_files)} images and labels from '{dataset_path}' to split '{split_type}'")

# Seed for reproducibility
random.seed(42)

# Combine datasets with the new split limits
for split, num_images in SPLITS.items():
    for idx, dataset in enumerate(SOURCE_DATASETS):
        select_and_copy(idx, dataset, split, num_images, split)

logging.info("Dataset combination complete.")

loading Roboflow workspace...
loading Roboflow project...


INFO: Downloading dataset 'ruby-ball-detection' version 3...
INFO: Downloaded to 'C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\unibots\src\Ruby-Ball-Detection-3'


loading Roboflow workspace...
loading Roboflow project...


INFO: Downloading dataset 'ping-pong-detection-0guzq' version 3...
INFO: Downloaded to 'C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\unibots\src\Ping-Pong-Detection-3'
INFO: Merged data.yaml created at C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Computer_Vision\datasets\data.yaml
INFO: Copied 550 images and labels from 'C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\unibots\src\Ruby-Ball-Detection-3' to split 'train'
INFO: Copied 550 images and labels from 'C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\unibots\src\Ping-Pong-Detection-3' to split 'train'
INFO: Copied 50 images and labels from 'C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\unibots\src\Ruby-Ball-Detection-3' to split 'valid'
INFO: Copied 50 images and labels from 'C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\unibots\src\Ping-Pong-Detection-3' to split 'valid'
INFO: Copied 34 images and labels from 'C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\unibots\src\Ruby-Ball-Detection-3' to split 'tes

### Great Ball Dataset

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key=api_key)
project = rf.workspace("zmsi").project("lab9-an4el")
version = project.version(1)
dataset = version.download("yolov11")


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/81.5 kB ? eta -:--:--
     ---------------------------------------- 81.5/81.5 kB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/54.5 kB ? eta -:--:--
     ---------------------------------------- 54.5/54.5 kB ? eta 0:00:00
     ---------------------------------------- 0.0/66.8 kB ? eta -:--:--
     ---------------------------------------- 66.8/66.8 kB ? eta 0:00:00
  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
     ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
      --------------------------------------- 0.9/38.8 MB 19.1 MB/s eta 0:00:02
     - -------------------------------------- 1.7/38.8 MB 21.1 MB/s eta 0:00:02
     -- ------------------------------------- 2.7/38.8 MB 19.0 MB/s eta 0:00:02
     --- ------------------------------------ 3.7/38.8 MB 19.7 MB/s eta 0:00:02
     ---- ----------------------------------- 4.8/38.8 MB 20.5 MB/s eta 0:00:02
     ------ --------------


Extracting Dataset Version Zip to lab9-1 in yolov11:: 100%|██████████| 2044/2044 [00:00<00:00, 2480.71it/s]


### Download WeBots Balls Dataset

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key=api_key2)
project = rf.workspace("5gkicker").project("webots-balls")
version = project.version(2)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to webots-balls-2 in yolov11:: 100%|██████████| 55/55 [00:00<00:00, 1412.83it/s]
